#Deep Learning Training (medical images of 6 classes)

Submission:
1. Project report (3-4 pages; include GitHub link).
2. GitHub with file(s).

#Importing all necessary libraries

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sklearn
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')


Mounted at /content/drive


#Importing the dataset (via unzipping from Google Colab)

In [3]:
# Unzipping the .zip file & saving it to the specified "/content/medical_data"
!unzip "/content/drive/MyDrive/Colab Notebooks/medical_data.zip" -d /content/medical_data

Streaming output truncated to the last 5000 lines.
  inflating: /content/medical_data/HeadCT/005000.jpeg  
  inflating: /content/medical_data/HeadCT/005001.jpeg  
  inflating: /content/medical_data/HeadCT/005002.jpeg  
  inflating: /content/medical_data/HeadCT/005003.jpeg  
  inflating: /content/medical_data/HeadCT/005004.jpeg  
  inflating: /content/medical_data/HeadCT/005005.jpeg  
  inflating: /content/medical_data/HeadCT/005006.jpeg  
  inflating: /content/medical_data/HeadCT/005007.jpeg  
  inflating: /content/medical_data/HeadCT/005008.jpeg  
  inflating: /content/medical_data/HeadCT/005009.jpeg  
  inflating: /content/medical_data/HeadCT/005010.jpeg  
  inflating: /content/medical_data/HeadCT/005011.jpeg  
  inflating: /content/medical_data/HeadCT/005012.jpeg  
  inflating: /content/medical_data/HeadCT/005013.jpeg  
  inflating: /content/medical_data/HeadCT/005014.jpeg  
  inflating: /content/medical_data/HeadCT/005015.jpeg  
  inflating: /content/medical_data/HeadCT/005016.jpeg

#Step 1: Preparing Data

In [4]:
# Directory path (we saved the .zip files, which contain 6 folders for the 6
# classes, to this earlier).
#
# AbdomenCT = 10,000
# BreastMRI = 8,954
# CXR = 10,000
# ChestCT = 10,000
# Hand = 10,000
# HeadCT = 10,000

# Path to your data directory (all images are under this directory).
data_dir = '/content/medical_data'

# Define ImageDataGenerator for training with augmentation and rescaling.
training_datagenerator = ImageDataGenerator(
    # Normalize pixel values to [0, 1].
    rescale=1./255,

    # Randomly rotate images.
    rotation_range=20,

    # Randomly shift images horizontally.
    width_shift_range=0.2,
    # Randomly shift images vertically.
    height_shift_range=0.2,

    # Randomly shear images.
    shear_range=0.2,
    # Random zoom.
    zoom_range=0.2,

    # Randomly flip images horizontally.
    horizontal_flip=True,

    # Fill pixels after transformation.
    fill_mode='nearest',

    # Split 20% of the data (from all folders) for validation.
    # Remaining 80% will be for training.
    validation_split=0.2
)

# Load training data (80% of data, with augmentation).
training_data = training_datagenerator.flow_from_directory(
    # Root directory containing class subdirectories.
    data_dir,

    # Resize all images to 64x64.
    target_size=(64, 64),
    # Number of images per batch.
    batch_size=32,

    # Multi-class classification.
    class_mode='categorical',
    # Use 80% of data for training.
    subset='training',
)


# Load validation data (20% of data, with no augmentation).
validation_data = training_datagenerator.flow_from_directory(
    # Root directory containing class subdirectories.
    data_dir,

    # Resize all images to 64x64.
    target_size=(64, 64),
    # Number of images per batch.
    batch_size=32,

    # Multi-class classification.
    class_mode='categorical',
    # Use 20% of data for validation.
    subset='validation',
)


Found 47164 images belonging to 6 classes.
Found 11790 images belonging to 6 classes.


#Step 2: Designing Model

In [5]:
# Import the pre-trained ResNet50 model from Keras.
from tensorflow.keras.applications import ResNet50

# Load the pre-trained ResNet50 model, excluding the top (classification) layer.
resnet_model = ResNet50(
    include_top=False,  # Exclude the fully connected layer (top)
    input_shape=(64, 64, 3),  # Resize input images to 64x64 RGB
    weights='imagenet'  # Use weights pre-trained on ImageNet
)

# Freeze the layers of the pre-trained model (ResNet50).
for layer in resnet_model.layers:
    layer.trainable = False

# Add a new classification head on top of ResNet50.
model = models.Sequential([
    resnet_model,
    layers.GlobalAveragePooling2D(),  # Global average pooling layer
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')  # Output layer for 6 classes
])

# Print model summary.
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2, 2, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,639,878 (93.99 MB)

 Trainable params: 1,052,166 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

#Step 3: Compile the Model


In [6]:
# Compile the model.
model.compile(
    # Adam optimizer.
    optimizer='adam',

    # Loss function for multi-class classification.
    loss='categorical_crossentropy',

    # Track accuracy.
    metrics=['accuracy']
)

#Step 4: Training the Model

In [ ]:
# Train the model.
history = model.fit(
    # Training data with augmentation.
    training_data,

    # Number of epochs.
    epochs=10,

    # Validation data.
    validation_data=validation_data
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1474/1474 ━━━━━━━━━━━━━━━━━━━━ 1082s 728ms/step - accuracy: 0.7206 - loss: 0.7589 - val_accuracy: 0.8932 - val_loss: 0.2974
Epoch 2/10
1474/1474 ━━━━━━━━━━━━━━━━━━━━ 1103s 729ms/step - accuracy: 0.8812 - loss: 0.3280 - val_accuracy: 0.9064 - val_loss: 0.2441
Epoch 3/10
1474/1474 ━━━━━━━━━━━━━━━━━━━━ 1065s 722ms/step - accuracy: 0.9003 - loss: 0.2704 - val_accuracy: 0.9171 - val_loss: 0.2147
Epoch 4/10
1474/1474 ━━━━━━━━━━━━━━━━━━━━ 1067s 724ms/step - accuracy: 0.9070 - loss: 0.2482 - val_accuracy: 0.9301 - val_loss: 0.1938
Epoch 5/10
1474/1474 ━━━━━━━━━━━━━━━━━━━━ 1070s 725ms/step - accuracy: 0.9121 - loss: 0.2331 - val_accuracy: 0.9318 - val_loss: 0.1872
Epoch 6/10
 113/1474 ━━━━━━━━━━━━━━━━━━━━ 13:08 579ms/step - accuracy: 0.9205 - loss: 0.2177

#Step 5: Evaluate the Model

In [ ]:
# Evaluate the model on validation data.
val_loss, val_accuracy = model.evaluate(validation_data)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")